The [day 11 puzzle](https://adventofcode.com/2021/day/11) involves analyzing a grid to increment values on each turn and also when one value reaches past 9 and "flashes".

     - First, each value increases by 1
     - Then, any value that exceeds 9 "flashes", raising the value of all neighboring values by 1
     - One value "flashing" another may cause it to exceed 9 and subsequently trigger more flashes
     - A value can only flash once per turn
     - On the next turn, any value that flashed is reset to 0
     
The first challenge is to count how many flashes happen by turn 100.  The second challenge is to observe when the flashes begin to synchronize and identify at what turn all values flash simultaneously.
     

In [1]:
import gridthings
import pandas  # only for printing out the grid, not for analysis

In [2]:
data = """
5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526
"""
# Using a custom Cell class to both cast each value to int
# and also make available a .has_flashed boolean on each cell
class FlashCell(gridthings.IntCell):
    has_flashed: bool = False


grid = gridthings.IntGrid(data, cell_cls=FlashCell)
grid

<IntGrid shape=(10, 10)>

In [3]:
pandas.DataFrame(grid.values())

,0,1,2,3,4,5,6,7,8,9
0,5,4,8,3,1,4,3,2,2,3
1,2,7,4,5,8,5,4,7,1,1
2,5,2,6,4,5,5,6,1,7,3
3,6,1,4,1,3,3,6,1,4,6
4,6,3,5,7,3,8,5,4,7,8
5,4,1,6,7,5,2,4,6,4,5
6,2,1,7,6,8,4,1,7,2,1
7,6,8,8,2,8,8,1,1,3,4
8,4,8,4,6,8,4,8,5,5,4
9,5,2,8,3,7,5,1,5,2,6


In [4]:
# As noted above, there's three distinct parts to each "turn" in this puzzle
# 1. Increment all values by 1
# 2. Handle flashes (1 flash/turn, keep track of total flashes)
#   2a. All cells that exceeded value 9 in step 1 flash
#   2b. Increment values on neighbors of those flashed cells
#   2c. Check if those new increments caused new flashes
#   2d. repeat until no new flashes happen
# 3. Reset any flashed cell value to 0, this completes a turn


class Solver:
    def __init__(self, data: str):
        self.grid = gridthings.Grid(data, cell_cls=FlashCell)
        self.flashes = 0
        self.turns = 0

    def step(self):
        # 1. increment values
        for cell in self.grid.flatten():
            cell.value += 1
        # 2. handle flashes
        while True:
            to_flash = []
            for cell in self.grid.flatten():
                if cell.value > 9 and not cell.has_flashed:
                    to_flash.append(cell)
            if to_flash:
                for cell in to_flash:
                    self.flash(cell)
            else:
                break
        # 3. Reset flashed cells
        self.reset_flashes()
        self.turns += 1

    def flash(self, cell):
        self.flashes += 1
        cell.has_flashed = True
        for neighbor in self.grid.peek_all(cell.y, cell.x):
            if isinstance(neighbor, FlashCell):
                neighbor.value += 1

    def reset_flashes(self):
        for cell in self.grid.flatten():
            if cell.has_flashed:
                cell.has_flashed = False
                cell.value = 0


solver = Solver(data)
solver

In [5]:
# Reminder of what the grid looks like initially
pandas.DataFrame(solver.grid.values())

,0,1,2,3,4,5,6,7,8,9
0,5,4,8,3,1,4,3,2,2,3
1,2,7,4,5,8,5,4,7,1,1
2,5,2,6,4,5,5,6,1,7,3
3,6,1,4,1,3,3,6,1,4,6
4,6,3,5,7,3,8,5,4,7,8
5,4,1,6,7,5,2,4,6,4,5
6,2,1,7,6,8,4,1,7,2,1
7,6,8,8,2,8,8,1,1,3,4
8,4,8,4,6,8,4,8,5,5,4
9,5,2,8,3,7,5,1,5,2,6


In [6]:
# After one step -- no flashes here
solver.step()
pandas.DataFrame(solver.grid.values())

,0,1,2,3,4,5,6,7,8,9
0,6,5,9,4,2,5,4,3,3,4
1,3,8,5,6,9,6,5,8,2,2
2,6,3,7,5,6,6,7,2,8,4
3,7,2,5,2,4,4,7,2,5,7
4,7,4,6,8,4,9,6,5,8,9
5,5,2,7,8,6,3,5,7,5,6
6,3,2,8,7,9,5,2,8,3,2
7,7,9,9,3,9,9,2,2,4,5
8,5,9,5,7,9,5,9,6,6,5
9,6,3,9,4,8,6,2,6,3,7


In [7]:
# In the next step, flashes begin happening
solver.step()
pandas.DataFrame(solver.grid.values())

,0,1,2,3,4,5,6,7,8,9
0,8,8,0,7,4,7,6,5,5,5
1,5,0,8,9,0,8,7,0,5,4
2,8,5,9,7,8,8,9,6,0,8
3,8,4,8,5,7,6,9,6,0,0
4,8,7,0,0,9,0,8,8,0,0
5,6,6,0,0,0,8,8,9,8,9
6,6,8,0,0,0,0,5,9,4,3
7,0,0,0,0,0,0,7,4,5,6
8,9,0,0,0,0,0,0,8,7,6
9,8,7,0,0,0,0,6,8,4,8


In [8]:
solver.turns, solver.flashes

(2, 35)

In [9]:
solver.step()
pandas.DataFrame(solver.grid.values())

,0,1,2,3,4,5,6,7,8,9
0,0,0,5,0,9,0,0,8,6,6
1,8,5,0,0,8,0,0,5,7,5
2,9,9,0,0,0,0,0,0,3,9
3,9,7,0,0,0,0,0,0,4,1
4,9,9,3,5,0,8,0,0,6,3
5,7,7,1,2,3,0,0,0,0,0
6,7,9,1,1,2,5,0,0,0,9
7,2,2,1,1,1,3,0,0,0,0
8,0,4,2,1,1,2,5,0,0,0
9,0,0,2,1,1,1,9,0,0,0


In [10]:
# The advent of code page says there should be 204 flashes after 10 turns
while solver.turns < 10:
    solver.step()
solver.flashes

204

In [11]:
# And 1656 after 100 turns
while solver.turns < 100:
    solver.step()
solver.flashes

1656

In [12]:
# second part of the challenge is to find when all values have flashed simultaneously
# which would mean the values in the grid are all 0s
#
# we can check for that by summing up the flattened grid
sum(solver.grid.flatten())

482

In [13]:
# According to the advent of code page, this will happen on the 195th turn
while True:
    if sum(solver.grid.flatten()):
        solver.step()
    else:
        break
solver.turns

195

In [14]:
pandas.DataFrame(solver.grid.values())

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0
